# Теоретические вопросы

*1) Какую роль выполняет функция активации в полносвязной нейросети? Также нужно пояснить, почему стакать несколько подряд идущих полносвязных слоев без активации не является хорошей практикой.* **0.5 балла**

*2) Допустим у вас есть MLP ("multi layer perceptron" или полносвязная нейросеть) с 10-ю входными нейронами, за ними 50 нейронов в скрытом слое, на выходе 3 нейрона. Все нейроны снабжены функцией активации ReLU.* **1 балл**
- Какова размерность матрицы $X$ на входе, если обучение проводится батчами. Размер батча $batch\_size$ [batchsize,10]
- Какова размерность матрицы весов скрытого слоя $W_h$ и какова размерность вектора байеса $b_h$ этого скрытого слоя? [10,50],[1,50]
- Какова размерность матрицы весов выходного слоя $W_o$ и какова размерность вектора байеса $b_o$ выходного слоя? [50,3],[1,3]
- Какова размерность выходной матрицы $Y$? - [batchsize,3]
- Напишите как выглядит функция, которая считает выходную матрицу $Y$ как функцию от $X$, $W_h$, $W_o$, $b_h$, $b_o$.

*3) Как устроено автоматическое дифференцирование в Tensorflow?* **0.5 балла**
- Используются ли символьные вычисления?
- Используется ли численное дифференцирование?
- Сколько проходов по графу вычислений необходимо совершить,
  чтобы посчитать градиенты для каждого параметра в Tensorflow?
  
*4) Что такое "backpropagation" и для чего он нужен. Чем он отличается от алгоритма градиентного спуска?* **0.5 балла**

*5) Расскажите в чем отличия между mini-batch, batch, stochastic gradient descent. Какой из этих алгоритмов самый популярный и почему.* **0.5 балла**

*6) Можно ли инициализировать первоначально все параметры обучаемой модели MLP единицами? А нулями? Поясните ваш ответ. Можно ли инициализировать нулями параметры $b_o$, $b_h$ из задачи №2?* **0.5 балла**

*7) Вывести лосс-функцию кросс-энтропии из правдоподобия и из теории информации.* **0.5 балла**

# Теоретические ответы

1) Функция активации определяет выходной сигнал с нейрона, в зависимости от поступивших входных сигналов. Стакать несколько слоев без функции активации бессмсленно, поскольку такую многослойную сеть можно заменить эквивалентной однослойной сетью.

2) размерность матрицы  X  на входе - [batchsize,10]

размерность матрицы весов скрытого слоя Wh и какова размерность вектора байеса  -  [10,50],[1,50]
  
[batchsize,50]
 
 размерность матрицы весов выходного слоя  Wo  и какова размерность вектора байеса  bo  выходного слоя [50,3],[1,3]

размерность выходной матрицы  Y [batchsize,3]


 
 Y = Wо * (Wh * X+bh)+bо


3) 

4) Backpropagation - это алгоритм построения вектора антиградиента. В нейронной сети мы не можем изменять сразу все веса, как например в однослойной нейронной цепи или линейной регрессии. Сперва необходимо определиться с тем, насколько необходимо подправить веса выходного слоя, потом на основании них подправить веса предыдущего слоя и так далее до весов входного слоя. Так формируется вектор антиградиента. А антиградиент это вектор в направлении которого изменяется вектор весов.

5) Batch это выборка объектов из датасета на одну итерацию. mini-batch это градиентный спуск, при расчете которого берется 1 батч а не вся выборка или только один объект. Количество батчей равно количеству итераций. размер батча(количество объектов в батче) умноженный на количество батчей равно количетсву объектов в датасете. Стохастический градиентный спуск - такой, при котором модель обучется на одном, случайно выбранном объекте. В нейронных сетях самый популярный - mini-batch. Позволяет значимо быстрее обучаться, по сравнению с обучением на полной выборке, и стаблиьнее себя ведет при обучении, не отклоняясь сильно из-за случайных выбросов, как мог бы при использовании стохастического градиентного спуска.

6) Если устанить все веса = 0 или = 1, производная по функции потерь будет одинакова для каждого  wi , таким образом, все веса имеют одинаковые значения и в последующей итерации. Это делает нейроны в сети симметричными. Так продолжается для всех n итераций. Установка смещений в 0 не вызовет каких-либо проблем, поскольку ненулевые веса позаботятся о нарушении симметрии, и даже если смещение равно 0, значения в каждом нейроне по-прежнему различаются.

7) 


# Первые шаги с TensorFlow

[Описание данных](https://developers.google.com/machine-learning/crash-course/california-housing-data-description)

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

C:\Users\1INVESTOR\Anaconda3\envs\py36\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


Подгружаем данные

In [2]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index)
)

california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
14510     -114.3      34.2                15.0       5612.0          1283.0   
5076      -114.5      34.4                19.0       7650.0          1901.0   
2806      -114.6      33.7                17.0        720.0           174.0   
14877     -114.6      33.6                14.0       1501.0           337.0   
4624      -114.6      33.6                20.0       1454.0           326.0   

       population  households  median_income  median_house_value  
14510      1015.0       472.0            1.5                66.9  
5076       1129.0       463.0            1.8                80.1  
2806        333.0       117.0            1.7                85.7  
14877       515.0       226.0            3.2                73.4  
4624        624.0       262.0            1.9                65.5

## EDA

Посмотрим на данные

In [3]:
california_housing_dataframe.describe()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
count    17000.0   17000.0             17000.0      17000.0         17000.0   
mean      -119.6      35.6                28.6       2643.7           539.4   
std          2.0       2.1                12.6       2179.9           421.5   
min       -124.3      32.5                 1.0          2.0             1.0   
25%       -121.8      33.9                18.0       1462.0           297.0   
50%       -118.5      34.2                29.0       2127.0           434.0   
75%       -118.0      37.7                37.0       3151.2           648.2   
max       -114.3      42.0                52.0      37937.0          6445.0   

       population  households  median_income  median_house_value  
count     17000.0     17000.0        17000.0             17000.0  
mean       1429.6       501.2            3.9               207.3  
std        1147.9       384.5            1.9               116.0  
min           3.0         1.0            0.5                15.0  
25%         790.0       282.0            2.6               119.4  
50%        1167.0       409.0            3.5               180.4  
75%        1721.0       605.2            4.8               265.0  
max       35682.0      6082.0           15.0               500.0

In [4]:
california_housing_dataframe.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value'],
      dtype='object')

## Первая модель

В этом задании требуется предсказать стоимость жилья (`median_house_value`).

Мы будем делать это с помощью [LinearRegressor](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearRegressor) и [tf estimators](https://www.tensorflow.org/get_started/estimator).

### 1. Определяем target

In [5]:
targets = california_housing_dataframe["median_house_value"]

### 2. Определяем входы в модель

С помощью [Dataset API](https://www.tensorflow.org/programmers_guide/datasets) и [Feature columns](https://www.tensorflow.org/guide/feature_columns) мы с легкостью определяем, как данные будут подаваться в модель.

In [6]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    
    # ключ total_rooms присутствует в features
    features = {key: np.array(value) for key, value in dict(features).items()}
 
    ds = tf.data.Dataset.from_tensor_slices((features, targets))
    # будем выдавать данные батчами в течение num_epochs эпох
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
        # делаем шафл только во время обучения
        # сможете ответить, зачем нужен шафл данных?
        ds = ds.shuffle(buffer_size=10000)

    return ds

my_feature = california_housing_dataframe[[ 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income']]
# по ключу "total_rooms" feature_column будет вытаскивать из tf.data.Dataset нужные фичи
feature_columns = [tf.feature_column.numeric_column('housing_median_age', shape=(1,)),
                   tf.feature_column.numeric_column("total_rooms", shape=(1,)),
                   tf.feature_column.numeric_column('total_bedrooms', shape=(1,)),
                   tf.feature_column.numeric_column('population', shape=(1,)),
                   tf.feature_column.numeric_column('households', shape=(1,)),
                   tf.feature_column.numeric_column('median_income', shape=(1,)),
                  ]

In [7]:
feature_columns

[_NumericColumn(key='housing_median_age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='total_rooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='total_bedrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='households', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='median_income', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

### 3. Определяем Модель

In [8]:
lr = 0.00001
my_optimizer = tf.train.AdamOptimizer(learning_rate=lr)

# посмотрите, какие аргументы принимает LinearRegressor
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

### 4. Обучаем модель

In [9]:
_ = linear_regressor.train(
    input_fn = lambda: my_input_fn(my_feature, targets),
    steps=100  # 1 step -- это проход по одному батчу
)

### 5. Считаем качество

In [10]:
prediction_input_fn = lambda: my_input_fn(my_feature, targets, num_epochs=1, shuffle=False)
# питоновский генератор
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# переводим в numpy
predictions = np.array([item['predictions'][0] for item in predictions])

# считаем метрики
mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = np.sqrt(mean_squared_error)

print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

Mean Squared Error (on training data): 55049.131
Root Mean Squared Error (on training data): 234.626


## Перебор гиперпараметров

In [11]:
def train_model(learning_rate, num_epochs, batch_size, input_feature="total_rooms"):

    my_feature_data = california_housing_dataframe[[input_feature]]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    feature_columns = [tf.feature_column.numeric_column(input_feature, shape=(1,))]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
    )

    print("Training model...")
    linear_regressor.train(input_fn=training_input_fn)

    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

## Задание 1 (2 балла)

Попробуйте поиграть с гиперпараметрами и достигнуть $RMSE \leq 180$

Также можно попробовать:
1. Другой оптимайзер ([здесь](https://www.tensorflow.org/api_docs/python/tf/train) можно посмотреть, какие бывают)
2. Комбинацию фичей (**feature_columns** -- это list, в котором несколько tf.feature_column.numeric_column)
3. Поиграть с аргументами модели (посмотрите, какие аргументы есть у LinearRegressor)

p.s.
Попробуйте переписать функцию `train_model`, чтобы было удобней изменять параметры модели.

In [12]:
# пример обучения модели
train_model(
    learning_rate=0.00001,
    num_epochs=5,
    batch_size=10
)

Training model...
Final RMSE (on training data): 171.43


In [13]:
# YOUR CODE HERE

## Задание 2 (2 балла)

Попробуйте подать в модель другие фичи и достигнуть $RMSE \leq 170$

In [14]:
input_features = [ 'housing_median_age', 'total_rooms','total_bedrooms', 'population', 'households', 'median_income']
def train_model(learning_rate, num_epochs, batch_size, input_feature=input_features):

    my_feature_data = california_housing_dataframe[input_feature]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    feature_columns = [tf.feature_column.numeric_column('housing_median_age', shape=(1,)),
                   tf.feature_column.numeric_column("total_rooms", shape=(1,)),
                   tf.feature_column.numeric_column('total_bedrooms', shape=(1,)),
                   tf.feature_column.numeric_column('population', shape=(1,)),
                   tf.feature_column.numeric_column('households', shape=(1,)),
                   tf.feature_column.numeric_column('median_income', shape=(1,)),
                  ]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
    )

    print("Training model...")
    linear_regressor.train(input_fn=training_input_fn)

    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

In [16]:
train_model(
    learning_rate=0.0001,
    num_epochs=10,
    batch_size=100
)

Training model...
Final RMSE (on training data): 165.78


## Задание 3 (2 балла)

Вместо `tf.estimator.LinearRegressor` попробуйте применить другую модель и достигнуть $RMSE \leq 160$

In [19]:
input_features = [ 'housing_median_age', 'total_rooms','total_bedrooms', 'population', 'households', 'median_income']
def train_model(learning_rate, num_epochs, batch_size, input_feature=input_features):

    my_feature_data = california_housing_dataframe[input_feature]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    feature_columns = [tf.feature_column.numeric_column('housing_median_age', shape=(1,)),
                   tf.feature_column.numeric_column("total_rooms", shape=(1,)),
                   tf.feature_column.numeric_column('total_bedrooms', shape=(1,)),
                   tf.feature_column.numeric_column('population', shape=(1,)),
                   tf.feature_column.numeric_column('households', shape=(1,)),
                   tf.feature_column.numeric_column('median_income', shape=(1,)),
                  ]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    hidden_layers = [16, 16]
    dropout = 0.2
    
    DNNregressor = tf.estimator.DNNRegressor(
                feature_columns=feature_columns,
                label_dimension=1,
                hidden_units=hidden_layers,
                dropout=dropout,
             
    )

    print("Training model...")
    DNNregressor.train(input_fn=training_input_fn)

    predictions = DNNregressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

In [20]:
train_model(
    learning_rate=0.00001,
    num_epochs=10,
    batch_size=20
)

Training model...
Final RMSE (on training data): 103.84
